# Cluster the Neighborhoods in Toronto: Part 3

In [3]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim

## From Parts 1 & 2

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto'
result = requests.get(url)
df = pd.DataFrame(columns=['Hood', 'Latitude', 'Longitude'])
df.head()

,Hood,Latitude,Longitude


In [6]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
lis = table.find_all('li')

list_of_n = []
for li in lis:
    a = li.find('a')
    list_of_n.append(a.get('title').split(", ")[0].split(" (neighbourhood)")[0].split(" (Toronto)")[0] )

In [8]:
df['Hood'] = pd.Series(list_of_n)
print(df.shape)
df.head()
df.drop_duplicates(inplace=True)
print(df.shape)
df.head()

(90, 3)
(87, 3)


,Hood,Latitude,Longitude
0,Alexandra Park,NaN,NaN
1,The Annex,NaN,NaN
2,Baldwin Village,NaN,NaN
3,Cabbagetown,NaN,NaN
4,CityPlace,NaN,NaN


In [10]:
to_drop_unknown = []
geolocator = Nominatim(user_agent="coursera")
for index, row in df.iterrows():
    address = row['Hood'] + ', Toronto'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df.loc[index, 'Latitude'] = latitude
        df.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

df.head()
clean_df = df.drop(to_drop_unknown)
clean_df.shape

The geograpical coordinate of Alexandra Park, Toronto are 43.65075755, -79.4042978683821.
The geograpical coordinate of The Annex, Toronto are 43.6703377, -79.407117.
The geograpical coordinate of Baldwin Village, Toronto are 43.66981815, -79.4971720154081.
The geograpical coordinate of Cabbagetown, Toronto are 43.6644734, -79.3669861.
The geograpical coordinate of CityPlace, Toronto are 43.6392482, -79.3963865.
The geograpical coordinate of Chinatown, Toronto are 43.6529237, -79.3980316.
The geograpical coordinate of Church and Wellesley, Toronto are 43.6655242, -79.3838011.
The geograpical coordinate of Corktown, Toronto are 43.6573709, -79.3565189.
The geograpical coordinate of Discovery District, Toronto are 43.6575555, -79.3894803.
The geograpical coordinate of Distillery District, Toronto are 43.6502947, -79.3595401.
The geograpical coordinate of Toronto Entertainment District, Toronto are 43.64383755, -79.3866924741406.
Cannot do: East Bayfront, Toronto, will drop index: 11
The 

(83, 3)

In [14]:
!pip install folium # Standard way
!conda install folium # If you use Conda
import folium

address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
    df.loc[index, 'Latitude'] = latitude
    df.loc[index, 'Longitude'] = longitude
except AttributeError:
    print('Cannot do: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(clean_df['Latitude'], clean_df['Longitude'], clean_df['Hood']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

    100% |████████████████████████████████| 92kB 18.4MB/s ta 0:00:01
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - folium

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/free/linux-64
  - https://repo.anaconda.com/pkgs/free/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/pro/linux-64
  - https://repo.anaconda.com/pkgs/pro/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


The geograpical coordinate of Toronto are 43.653963, -79.387207.
